## Making csv file

In [52]:
import codecs
from bs4 import BeautifulSoup
import csv
import os

In [62]:
csv_file.close()

In [63]:
csv_file = open('dou_vacancies.csv', 'w', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['headline', 'description'])

22

In [64]:
directory = 'dou/rezult'
for filename in os.listdir(directory):    
    file = codecs.open(directory + '/' + filename, "r", 'utf-8')
    soup = BeautifulSoup(file)
    headline = soup.find('h1', class_="g-h2").get_text()
    vacancy = soup.find('div', class_="l-vacancy")
    descriptions = vacancy.find_all('div', class_="text b-typo vacancy-section")
    text = ''
    for description in descriptions:
        text += description.get_text()
    csv_writer.writerow([headline, text])

In [ ]:
import pandas as pd

In [65]:
df = pd.read_csv('dou_vacancies.csv')

In [67]:
df.head()

,headline,description
0,"Junior ML engineer (Computer Vision, 3d)",\r\n— Python— хорошая математическая база для ...
1,Senior Java/BigData Engineer,\r\nExperience in Java development (5 years+);...
2,Java Engineer for HighLoad/BigData project | SM,\r\n— Professional Java experience 3+ years— O...
3,NLP Engineer,"\r\n— 2+ years experience with Python, includi..."
4,Data Engineer,\r\n— 2+ years of hands-on experience as a Dat...


## Text processing

In [103]:
import re
import nltk
from nltk.tokenize import word_tokenize
#nltk.download('stopwords')
#nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from string import punctuation
punctuation += '—'

In [90]:
stop = set(stopwords.words('english'))
punct = list(punctuation)

In [83]:
def clean_text(text):
    text = text.replace(u'\xa0', u' ')
    text_eng = re.sub('[^a-zA-Z0-9-+ ]+', '', text)
    return text_eng

In [92]:
def remove_stop_words(text):
    lst = word_tokenize(text)
    lst = [word for word in lst if word not in stop and word not in punct]
    return lst

In [123]:
def leave_nouns(lst):
    new_lst = [word for (word,pos) in nltk.pos_tag(lst) if pos[:2] == 'NN']
    return new_lst

In [99]:
df['description'] = df.description.apply(clean_text)

In [113]:
df['description'] = df.description.apply(remove_stop_words)

In [125]:
df['description'] = df.description.apply(leave_nouns)

In [119]:
df['is_data_engineer'] = df['headline'].apply(lambda x: 'data' in x.lower())

In [132]:
df.head()

,headline,description,is_data_engineer
0,"Junior ML engineer (Computer Vision, 3d)","[Python, machine, learning, Numpy, Pandas, Sci...",False
1,Senior Java/BigData Engineer,"[Experience, Java, development, years+Experien...",True
2,Java Engineer for HighLoad/BigData project | SM,"[Java, experience, years, OOPdesign, proficien...",True
3,NLP Engineer,"[years, Python, engineering, experience, API, ...",False
4,Data Engineer,"[years, experience, Data, Engineer, experience...",True


In [133]:
df['is_data_engineer'].value_counts()

True     118
False     38
Name: is_data_engineer, dtype: int64

In [139]:
df_engineers = df.query('is_data_engineer == True').copy()

In [140]:
df_engineers['description']

1      [Experience, Java, development, years+Experien...
2      [Java, experience, years, OOPdesign, proficien...
4      [years, experience, Data, Engineer, experience...
5      [Clients, mission, everyone, eat, Company, mea...
6      [years, Scala, development, cloud, services, A...
                             ...                        
151    [Experience, data, analytics, data, engineerin...
152    [cross-platformcross-browser, QA, solution, do...
153    [Fractal, Kyiv, team, position, junior, data, ...
154    [years, experience, software, development, pri...
155    [Job, SummaryDevelop, Rokus, edge, delivery, i...
Name: description, Length: 118, dtype: object

In [141]:
df_engineers['text'] = df_engineers['description'].apply(lambda x : " ".join(x))

In [144]:
all_text = df_engineers['text'].str.cat(sep=', ')

In [149]:
import spacy
from spacy.lang.en import English
nlp = spacy.load('en_core_web_md')

In [150]:
doc = nlp(all_text)

In [162]:
list_technologies = []

In [163]:
for ent in doc.ents:
    if ent.label_ != 'DATE' and ent.label_ != 'GPE':        
        list_technologies.append(ent.text)

In [166]:
set_of_technologies = set(list_technologies)

In [174]:
set_of_technologies

{'AI',
 'AIML',
 'ALM',
 'AREPassionate',
 'AWS',
 'AWS Azure GCP Airflow',
 'AWS Azure GCP Knowledge',
 'AWS Azure GCPData',
 'AWS Azure SaaS',
 'AWS Big Data Data Engineer AWS Data Lake Distributed Data Mesh Python',
 'AWS Cloud Process',
 'AWS EMR Lambda S3',
 'AWS Expertise',
 'AWS GCP',
 'AWS GCP Ability',
 'AWS GCP AzureCommercial',
 'AWS GCP BigQueryBe Object',
 'AWS Good',
 'AWS QuickSight Google Data',
 'AWS Redshift',
 'AWS Step',
 'AWS Strong',
 'AWSWorking Conditions Premium',
 'Administer Bigquery',
 'AgensGraph',
 'Agile ScrumGood',
 'Agile Software Development',
 'Aiohttp FastAPI',
 'Airflow Argo Workflows Kubeflow',
 'AkkaExperience',
 'Allianz Group',
 'Amazon',
 'Amazon Athena',
 'Amazon Athena Apache',
 'Amazon Redshift',
 'Amazon Redshift Big Query Ability',
 'Amazon Redshift DBMS SQL',
 'Amazon RedshiftExperience',
 'Amazon Web Services',
 'Amazons',
 'Analysis',
 'Analysis Interpretation Validation',
 'Analytics Data',
 'Andreessen Horowitz',
 'Ansible Kubernetes'

In [167]:
len(set_of_technologies)

752

In [196]:
for ent in doc.ents:
    #if ent.label_ == 'CARDINAL':
    if ent.label_ != 'DATE' and ent.label_ != 'GPE' and ent.label_ != 'LOC' and ent.label_!= 'CARDINAL' and ent.label_ != 'TIME': 
        print(ent.text, ent.label_)

HadoopTechnical ORG
Jenkins CICD PERSON
Intellias PERSON
Personal Development PlanRegular ORG
Lviv KyivFor PERSON
Lviv KyivSolve ORG
Big Data ORG
SQL ORG
standardsUse ORG
SQLNoSQL PRODUCT
Familiarity Adtech ORG
Familiarity Aerospike ORG
Xenoss PERSON
Data Engineer ORG
Machine Learning Data Science Familiar ML Experience ORG
S3 ORG
ETL ORG
BI ORG
Looker ORG
AWS QuickSight Google Data ORG
Fridge ORG
SQL ORG
dbtWe ORG
Healthcare ORG
Apache Spark Kafka PRODUCT
SQL ORG
SQL BigData ORG
English NORP
Experience CICD Knowledge ORG
Vystavkovyi PRODUCT
UI ORG
Data WarehouseData Integration Knowledge ORG
Kimball Inmon Hands- ORG
Business IntelligenceAnalytical ORG
Tableau Extensive ORG
SQL ORG
DBMS ORG
Amazon Redshift ORG
GPC ORG
WFH ORG
AWS ORG
ETL ORG
Scala Java PERSON
Elasticsearch Hadoop MongoDB ORG
Kafka Zookeeper Docker ORG
Linux Bash PERSON
Ansible Kubernetes PERSON
CICD ORG
PaaSSaaS ORG
US Dedicated HR Client ORG
Big Data Infrastructure ORG
second ORDINAL
Scala ORG
Data Engineer ORG
Busine

In [197]:
#final_list.sort()
techno_list = ['AWS', 'Apache', 'Asure', 'Bash', 'BigQuery', 'Cassandra', \
              'Databricks', 'Delta Lake', 'Docker', 'ETL', 'Fiddler', 'GCP', 'Jenkins', 'Jira', 'JSON',\
              'Kafka', 'Kinesis', 'Linux', 'Mesos', 'MongoDB', 'MySQLOracle', 'NoSQL',\
              'Postman', 'PowerBI', 'PySpark', 'QA', 'R-Studio', 'Rabbit', 'ReactJS',\
              'SQL', 'Tableau', 'Tensorflow', 'XML']

In [198]:
prog_lang = ['Scala' 'Java', 'JS', 'JavaScript', 'Python']

In [199]:
soft_skills = ['Scrum', 'agile']